This is to test the workflow and printing of the images by using the following two tutorials:

[PyTorch Vizualization Tutorial](https://github.com/Project-MONAI/tutorials/blob/main/3d_segmentation/spleen_segmentation_3d_visualization_basic.ipynb)

[PyTorch Lightning Tutorial](https://github.com/Project-MONAI/tutorials/blob/main/3d_segmentation/spleen_segmentation_3d_lightning.ipynb)

Config

In [1]:
import torch
import torch.nn as nn
#import albumentations as A
import numpy as np
import time
import os

"""
Fistula Segmentation
"""
class Configuration:
    def __init__(self):
        self.init = {
            'PROJECT_NAME': 'FistulaSegmentation',
            'MODEL_NAME': 'Development',
            'RUN_NAME': time.strftime('%Y-%m-%d-%H-%M-%S'),
            'WANDB_RUN_GROUP': 'Local',
            'FAST_DEV_RUN': True,  # Runs inputted batches (True->1) and disables logging and some callbacks
            'MAX_EPOCHS': 3,
            'MAX_STEPS': -1,    # -1 means it will do all steps and be limited by epochs
            'STRATEGY': 'auto'    # This is the training strategy. Should be 'ddp' for multi-GPU (like HPG)
        }
        self.etl = {
            'DATA_DIR': "data",
            # Lol what is this?
            # HHG2TG lol; deterministic to aid reproducibility
            'RANDOM_STATE': 42,
        }

        self.dataset = {
            'DATA_NAME': 'BaseSplit',
            'USE_TRANSFORMS': False,
            'IMAGE_ROOT': '/media/sasank/LinuxStorage/Dropbox (UFL)/FistulaData/Segmentations/',
            'IMAGE_SIZE': (512, 512, 96)
        }

        self.datamodule = {
            'CKPT_FILE': '/media/sasank/LinuxStorage/Dropbox (UFL)/FistulaData/checkpoints/100Epoch87IOU.ckpt',
            'BATCH_SIZE': 1,
            'FIT_CACHE_NUM': 2,
            'SHUFFLE': True,        # Only for training; for test and val this is set in the datamodule script to False
            'NUM_WORKERS': 2,
            'PIN_MEMORY': False
        }


        # hyperparameters for training
        self.hparams = {
            'LOAD_FROM_CHECKPOINT': False,
            'learning_rate': 1e-3
        }

        """
        #self.transform = None
        self.transform = \
        A.Compose([
            # Let's do only rigid transformations for now
            A.HorizontalFlip(p=0.2),
            A.VerticalFlip(p=0.2),
            A.RandomRotate90(p=0.2),
            A.Transpose(p=0.2),
        ],
        p=1.0)
        """


In [ ]:
config = Configuration()

Dataset (let's just do Dataset and not do Datamodule yet)

In [ ]:
#self.config = config
batch_size = config.datamodule['BATCH_SIZE']
#self.log(batch_size)
num_workers = config.datamodule['NUM_WORKERS']
pin_memory = config.datamodule['PIN_MEMORY']
shuffle = config.datamodule['SHUFFLE']

# TODO: check train dataset length and integrity
# TODO: check val dataset length and integrity

# Create train, val, test splits of the data
self.train_data = pd.read_csv(os.path.join(self.config.etl['DATA_DIR'], self.config.dataset['DATA_NAME'], 'train' + '_' + self.config.dataset['DATA_NAME'] + '.csv'))
self.val_data = pd.read_csv(os.path.join(self.config.etl['DATA_DIR'], self.config.dataset['DATA_NAME'], 'val' + '_' + self.config.dataset['DATA_NAME'] + '.csv'))
self.test_data = pd.read_csv(os.path.join(self.config.etl['DATA_DIR'], self.config.dataset['DATA_NAME'], 'test' + '_' + self.config.dataset['DATA_NAME'] + '.csv'))

resample_model = os.path.join(self.config.dataset['IMAGE_ROOT'], '7/7_image.nii.gz')

# For each splits, create a list of dictionaries containing the image paths, label paths, and patient IDs
# Have both image and label paths prepended by the self.config.dataset['IMAGE_ROOT']
self.train_files = [{'image': os.path.join(self.config.dataset['IMAGE_ROOT'], self.train_data.iloc[i]['image']),
                'label': os.path.join(self.config.dataset['IMAGE_ROOT'], self.train_data.iloc[i]['label']),
                'resample_model': resample_model,
                'patient_id': self.train_data.iloc[i]['patient_id']} for i in range(len(self.train_data))]
self.val_files = [{'image': os.path.join(self.config.dataset['IMAGE_ROOT'], self.val_data.iloc[i]['image']),
                'label': os.path.join(self.config.dataset['IMAGE_ROOT'], self.val_data.iloc[i]['label']),
                'resample_model': resample_model,
                'patient_id': self.val_data.iloc[i]['patient_id']} for i in range(len(self.val_data))]
self.test_files = [{'image': os.path.join(self.config.dataset['IMAGE_ROOT'], self.test_data.iloc[i]['image']),
                'label': os.path.join(self.config.dataset['IMAGE_ROOT'], self.test_data.iloc[i]['label']),
                'resample_model': resample_model,
                'patient_id': self.test_data.iloc[i]['patient_id']} for i in range(len(self.test_data))]